## "Исследование поведения пользователей в игре"

In [1]:
import pandas as pd

Чтение данных:

In [2]:
events_data = pd.read_csv('Events.csv', sep=',')
purchases_data = pd.read_csv('purchase.csv', sep=',')

events_df = events_data.copy()
purchase_df = purchases_data.copy()

Оставляем в таблицах только те данные пользователей, которые были зарегистрированы в 2018 году:

In [3]:
mask_date = (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] < '2019-01-01') & (events_df['event_type'] == 'registration')
id_series = events_df[mask_date]['user_id']

mask_user = events_df['user_id'].isin(id_series)
events_df = events_df[mask_user]

mask2_user = purchase_df['user_id'].isin(id_series)
purchase_df = purchase_df[mask2_user]

Преобразуем столбец по выбору сложностей в тип категорий и узнаем возможные уровни сложностей:

In [4]:
events_df['selected_level'] = events_df['selected_level'].astype('category')

print(events_df['selected_level'].cat.categories)

Index(['easy', 'hard', 'medium'], dtype='object')


Создадим переменные, которые будут хранить группы пользователей (в зависимости от выбранной сложности) и их количество:

In [5]:
easy_users_count = events_df[events_df["selected_level"] == "easy"]["user_id"].nunique()
medium_users_count = events_df[events_df["selected_level"] == "medium"]["user_id"].nunique()
hard_users_count = events_df[events_df["selected_level"] == "hard"]["user_id"].nunique()

easy_users = events_df[events_df["selected_level"] == "easy"]["user_id"]
medium_users = events_df[events_df["selected_level"] == "medium"]["user_id"]
hard_users = events_df[events_df["selected_level"] == "hard"]["user_id"]

Рассчитаем процент пользователей, которые оплатили вопросы, от числа пользователей отпределённых групп:

In [6]:
paying_users_count_easy = purchase_df[purchase_df['user_id'].isin(easy_users)]["user_id"].nunique()
percent_of_easy_users = paying_users_count_easy / easy_users_count
print(
    "\nПроцент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности 'easy'): {:.2%}".format(
        percent_of_easy_users
    )
)

paying_users_count_medium = purchase_df[purchase_df['user_id'].isin(medium_users)]["user_id"].nunique()
percent_of_medium_users = paying_users_count_medium / medium_users_count
print(
    "\nПроцент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности 'medium'): {:.2%}".format(
        percent_of_medium_users
    )
)

paying_users_count_hard = purchase_df[purchase_df['user_id'].isin(hard_users)]["user_id"].nunique()
percent_of_hard_users = paying_users_count_hard / hard_users_count
print(
    "\nПроцент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности 'hard'): {:.2%}".format(
        percent_of_hard_users
    )
)


Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности 'easy'): 7.72%

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности 'medium'): 20.86%

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности 'hard'): 35.39%


По показаетлю оплаты видно, что пользователи, которые выбрали уровень сложности 'hard', с большей вероятностью оплатят вопросы в игре.

Объединяем таблицы событий и оплат:

In [7]:
"""Добавим в датафрейм purchase_df столбец event_type, который будет содержать одно значение purchase. 
Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты."""

purchase_df['event_type'] = 'purchase'

"""Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их несколько отличается, 
так как столбец id в events_df указывает на идентификатор события, 
а столбец id в purchase_df указывает на идентификатор оплаты. 
Поэтому применим функцию rename(), чтобы переименовать столбцы в датафреймах."""

events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

"""объединим датафреймы events_df и purchase_df с помощью функции pd.concat() и запишем объединённый датафрейм в total_events_df."""

total_events_df = pd.concat([events_df,purchase_df],sort=False)

"""Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) 
с помощью метода reset_index() и отсортируем все события по возрастанию времени с помощью sort_values()."""

total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

In [8]:
#for 'easy'

level_df_easy = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(easy_users))]
level_df_easy = level_df_easy[["user_id", "start_time"]].rename(columns={"start_time": "level_choice_time"})

purchase_df_easy = total_events_df[(total_events_df["event_type"] == "purchase") & (total_events_df['user_id'].isin(easy_users))]
purchase_df_easy = purchase_df_easy[["user_id", "event_datetime"]].rename(columns={"event_datetime": "purchase_time"})

merged_df_easy = level_df_easy.merge(purchase_df_easy, on="user_id", how="inner")
merged_df_easy["timedelta_level_purchase"] = (pd.to_datetime(merged_df_easy["purchase_time"]) - pd.to_datetime(merged_df_easy["level_choice_time"]))

registration_df_easy = total_events_df[(total_events_df['event_type'] == 'registration') & (total_events_df['user_id'].isin(easy_users))]
registration_df_easy = registration_df_easy[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})

merged_df_easy_reg = merged_df_easy.merge(registration_df_easy, on="user_id", how="inner")
merged_df_easy_reg['timedelta_reg_purchase'] = (pd.to_datetime(merged_df_easy_reg["purchase_time"]) - pd.to_datetime(merged_df_easy_reg["registration_time"]))

print('\nСрез таблицы для группы пользователей, выбравших уровень сложности easy:')
print('\n',merged_df_easy_reg.head())
print('\nСреднее время между событиями оплаты и выбором уровня сложности timedelta_level_purchase:')
print(merged_df_easy_reg["timedelta_level_purchase"].mean())
print('\nСреднее время между событиями регистрации и оплаты timedelta_reg_purchase:')
print(merged_df_easy_reg["timedelta_reg_purchase"].mean())


Срез таблицы для группы пользователей, выбравших уровень сложности easy:

    user_id    level_choice_time        purchase_time timedelta_level_purchase  \
0    27884  2018-01-04T16:18:39  2018-01-08T19:37:34          4 days 03:18:55   
1    28090  2018-01-09T21:34:23  2018-01-15T23:42:55          6 days 02:08:32   
2    28182  2018-01-11T18:44:45  2018-01-12T02:46:01          0 days 08:01:16   
3    28207  2018-01-11T21:10:51  2018-01-12T21:00:24          0 days 23:49:33   
4    28254  2018-01-12T16:48:24  2018-01-19T22:08:40          7 days 05:20:16   

     registration_time timedelta_reg_purchase  
0  2018-01-04T11:50:43        4 days 07:46:51  
1  2018-01-09T19:31:24        6 days 04:11:31  
2  2018-01-11T10:12:20        0 days 16:33:41  
3  2018-01-11T16:27:37        1 days 04:32:47  
4  2018-01-12T12:14:00        7 days 09:54:40  

Среднее время между событиями оплаты и выбором уровня сложности timedelta_level_purchase:
3 days 14:58:52.941798941

Среднее время между событиями р

In [9]:
#for medium

level_df_medium = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(medium_users))]
level_df_medium = level_df_medium[["user_id", "start_time"]].rename(columns={"start_time": "level_choice_time"})

purchase_df_medium = total_events_df[(total_events_df["event_type"] == "purchase") & (total_events_df['user_id'].isin(medium_users))]
purchase_df_medium = purchase_df_medium[["user_id", "event_datetime"]].rename(columns={"event_datetime": "purchase_time"})

merged_df_medium = level_df_medium.merge(purchase_df_medium, on="user_id", how="inner")
merged_df_medium["timedelta_level_purchase"] = (pd.to_datetime(merged_df_medium["purchase_time"]) - pd.to_datetime(merged_df_medium["level_choice_time"]))

registration_df_medium = total_events_df[(total_events_df['event_type'] == 'registration') & (total_events_df['user_id'].isin(medium_users))]
registration_df_medium = registration_df_medium[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})

merged_df_medium_reg = merged_df_medium.merge(registration_df_medium, on="user_id", how="inner")
merged_df_medium_reg['timedelta_reg_purchase'] = (pd.to_datetime(merged_df_medium_reg["purchase_time"]) - pd.to_datetime(merged_df_medium_reg["registration_time"]))

print('\nСрез таблицы для группы пользователей, выбравших уровень сложности medium:')
print('\n',merged_df_medium_reg.head())
print('\nСреднее время между событиями оплаты и выбором уровня сложности timedelta_level_purchase:')
print(merged_df_medium_reg["timedelta_level_purchase"].mean())
print('\nСреднее время между событиями регистрации и оплаты timedelta_reg_purchase:')
print(merged_df_medium_reg["timedelta_reg_purchase"].mean())


Срез таблицы для группы пользователей, выбравших уровень сложности medium:

    user_id    level_choice_time        purchase_time timedelta_level_purchase  \
0    27973  2018-01-07T05:29:30  2018-01-13T21:50:00          6 days 16:20:30   
1    27981  2018-01-07T10:46:14  2018-01-07T23:20:25          0 days 12:34:11   
2    28010  2018-01-08T00:00:52  2018-01-10T05:32:47          2 days 05:31:55   
3    28020  2018-01-08T14:47:35  2018-01-11T21:43:03          3 days 06:55:28   
4    28033  2018-01-08T17:06:39  2018-01-16T05:08:41          7 days 12:02:02   

     registration_time timedelta_reg_purchase  
0  2018-01-06T22:01:44        6 days 23:48:16  
1  2018-01-07T08:09:09        0 days 15:11:16  
2  2018-01-07T22:19:23        2 days 07:13:24  
3  2018-01-08T10:30:32        3 days 11:12:31  
4  2018-01-08T16:52:19        7 days 12:16:22  

Среднее время между событиями оплаты и выбором уровня сложности timedelta_level_purchase:
3 days 23:14:13.165118679

Среднее время между событиями

In [10]:
#for hard

level_df_hard = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(hard_users))]
level_df_hard = level_df_hard[["user_id", "start_time"]].rename(columns={"start_time": "level_choice_time"})

purchase_df_hard = total_events_df[(total_events_df["event_type"] == "purchase") & (total_events_df['user_id'].isin(hard_users))]
purchase_df_hard = purchase_df_hard[["user_id", "event_datetime"]].rename(columns={"event_datetime": "purchase_time"})

merged_df_hard = level_df_hard.merge(purchase_df_hard, on="user_id", how="inner")
merged_df_hard["timedelta_level_purchase"] = (pd.to_datetime(merged_df_hard["purchase_time"]) - pd.to_datetime(merged_df_hard["level_choice_time"]))

registration_df_hard = total_events_df[(total_events_df['event_type'] == 'registration') & (total_events_df['user_id'].isin(hard_users))]
registration_df_hard = registration_df_hard[["user_id", "start_time"]].rename(columns={"start_time": "registration_time"})

merged_df_hard_reg = merged_df_hard.merge(registration_df_hard, on="user_id", how="inner")
merged_df_hard_reg['timedelta_reg_purchase'] = (pd.to_datetime(merged_df_hard_reg["purchase_time"]) - pd.to_datetime(merged_df_hard_reg["registration_time"]))

print('\nСрез таблицы для группы пользователей, выбравших уровень сложности hard:')
print('\n',merged_df_hard_reg.head())
print('\nСреднее время между событиями оплаты и выбором уровня сложности timedelta_level_purchase:')
print(merged_df_hard_reg["timedelta_level_purchase"].mean())
print('\nСреднее время между событиями регистрации и оплаты timedelta_reg_purchase:')
print(merged_df_hard_reg["timedelta_reg_purchase"].mean())


Срез таблицы для группы пользователей, выбравших уровень сложности hard:

    user_id    level_choice_time        purchase_time timedelta_level_purchase  \
0    27845  2018-01-02T06:19:18  2018-01-03T18:53:43          1 days 12:34:25   
1    27865  2018-01-04T05:56:32  2018-01-04T14:46:10          0 days 08:49:38   
2    27910  2018-01-05T11:59:50  2018-01-07T12:11:34          2 days 00:11:44   
3    27911  2018-01-05T17:39:02  2018-01-07T08:19:12          1 days 14:40:10   
4    27940  2018-01-06T00:32:47  2018-01-07T13:16:41          1 days 12:43:54   

     registration_time timedelta_reg_purchase  
0  2018-01-02T01:35:56        1 days 17:17:47  
1  2018-01-03T11:14:57        1 days 03:31:13  
2  2018-01-05T10:45:33        2 days 01:26:01  
3  2018-01-05T10:48:24        1 days 21:30:48  
4  2018-01-05T23:41:24        1 days 13:35:17  

Среднее время между событиями оплаты и выбором уровня сложности timedelta_level_purchase:
3 days 07:20:41.420814479

Среднее время между событиями р

Выводы о различии в показателях между группами:
- среднее время между событиями оплаты и выбором уровня сложности для всех групп составляет 3-4 дня. Чуть меньшее время - для группы пользователей, выбравших сложный уровень. 
- разница во времени между событиями регистрации и оплаты незначительно выше, чем между событиями оплаты и выбором уровня сложности. 

Учитывая, что пользователи, выбравшие уровень сложности 'hard' составляют наибольший процент пользователей, оплативших вопросы, и что им требуется меньшее время для решения о покупки вопросов, можно сделать вывод, что уровень сложности 'hard' наиболее заинтересовывает пользователей в игру. 
